In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

/tmp/ipykernel_2388219/1532855953.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [25]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={})]}

In [26]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [27]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={})]}

In [28]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [29]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [35]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [36]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions they live in South Korea.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kw